In [59]:
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import glob
import time
import os
from scipy.stats import shapiro, mannwhitneyu

In [60]:
FORM = "%H:%M:%S.%f"
DATASET = {"b": "wiki-talk", "c": "kgs", "d": "datagen7.5"}
TESTS = ["tenantb-pagerank", "tenantb-sp", "tenantc-pr", "tenantc-sp", "tenantb-tc", "tenantc-tc", "tenantc-lpa", "tenantb-lpa"]

BASE = "C:/Users/hzwaan/Documents/serverlessgraphs/timestamps/result_v2/raw"

In [61]:
runs = glob.glob(f"{BASE}/*.csv")
timings = glob.glob(f"{BASE}/*.txt")

In [62]:
runnings_df = None  
timings_df = None

In [63]:
runnings_df = None  
for i, run in enumerate(runs):
    run_df = pd.read_csv(run)
    csv_path = run.split("\\",1)[1]
    identifier = csv_path.split('.')[0]

    dataset, algorithm = identifier.split("-")[:2]
    dataset = DATASET[dataset[-1]]
    algorithm = algorithm if algorithm != "pagerank" else "pr"

    if len(run_df) != 6:
        continue

    date = run_df.iloc[2]["Date"]
    creation = datetime.strptime(run_df.iloc[1]["Time"], FORM) - datetime.strptime(run_df.iloc[0]["Time"], FORM)
    if run_df.iloc[3]["Date"] != run_df.iloc[2]["Date"]:
        execution = (datetime.strptime(run_df.iloc[3]["Time"], FORM) + timedelta(days=1)) - datetime.strptime(run_df.iloc[2]["Time"], FORM)
    else:
        execution = datetime.strptime(run_df.iloc[3]["Time"], FORM) - datetime.strptime(run_df.iloc[2]["Time"], FORM)
    deletion = datetime.strptime(run_df.iloc[5]["Time"], FORM) - datetime.strptime(run_df.iloc[4]["Time"], FORM)

    creation_sec = int(creation.total_seconds())
    execution_sec = int(execution.total_seconds())
    deletion_sec = int(deletion.total_seconds())
    total_sec = creation_sec + execution_sec + deletion_sec

    begin_tp = run_df.iloc[2]["Time"]
    end_tp = run_df.iloc[3]["Time"]
    h,m,s = end_tp.split(":")
    s,ms = s.split(".")
    s = f"{str(int(s) + 5)}.{ms}" if int(s) + 5 <= 60 else "60.999999"
    if s == "60.999999":
        m = str(int(m) + 1)
    end_tp = f"{h}:{m}:{s}"

    result = pd.DataFrame([creation, execution, deletion, creation_sec, execution_sec, deletion_sec, total_sec, identifier, date, begin_tp, end_tp])
    result.index = ["creation", "execution", "deletion", "creation_sec", "execution_sec", "deletion_sec", "total_sec", "identifier", "date", "begin_tp", "end_tp"]
    result = result.transpose()
    result["creation"] = result["creation"].astype(str).replace("0 days ", "", regex=True)
    result["execution"] = result["execution"].astype(str).replace("0 days ", "", regex=True)
    result["deletion"] = result["deletion"].astype(str).replace("0 days ", "", regex=True)
    result["dataset"] = dataset
    result["algorithm"] = algorithm
    if runnings_df is None:
        runnings_df = result
    else:
        runnings_df = runnings_df._append(result, ignore_index=True)

In [64]:
def date_format(time_stamp, day_offset = 0):
    if "day" in time_stamp:
        d, t = time_stamp.split("days")
        return datetime.strptime(t.strip(), FORM) + timedelta(days=(int(d) + day_offset))
    else:
        return datetime.strptime(time_stamp, FORM)

In [65]:
timings_df = None
for j, timing in enumerate(timings):
    execute = pd.read_csv(timing, sep=" ", header=None)
    execute.columns = ["Date", "Time"]
    execute.insert(0, "Action", ["start", "dataframe", "finished"])
    execute["Time"] = execute["Time"] + pd.Timedelta(hours=2)
    execute["Time"] = execute["Time"].astype(str).replace("0 days ", "", regex=True)
    identifier = timing.split("\\",1)[1]

    date = execute.iloc[1]["Date"]
    tp_dataframe_creation = date_format(execute.iloc[1]["Time"]) - date_format(execute.iloc[0]["Time"])

    if execute.iloc[2]["Date"] != execute.iloc[1]["Date"]:
        tp_execution = date_format(execute.iloc[2]["Time"], day_offset=1) - date_format(execute.iloc[1]["Time"])
    else:
        tp_execution = date_format(execute.iloc[2]["Time"]) - date_format(execute.iloc[1]["Time"])
        
    if "-" in str(tp_execution):
        tp_execution = tp_execution + timedelta(days=int(str(tp_execution).split("day")[0].replace("-", "")))


    tp_dataframe_sec = int(tp_dataframe_creation.total_seconds())
    tp_execution_sec = int(tp_execution.total_seconds())

    begin = execute.iloc[0]["Time"]
    end = execute.iloc[2]["Time"]

    result = pd.DataFrame([tp_dataframe_creation, tp_execution, tp_dataframe_sec, tp_execution_sec, date, begin, end, identifier])
    result.index = ["tp_dataframe", "tp_execution", "tp_dataframe_sec", "tp_execution_sec", "date", "begin", "end", "tp_identifier"]
    result = result.transpose()
    result["tp_dataframe"] = result["tp_dataframe"].astype(str).replace("0 days ", "", regex=True)
    result["tp_execution"] = result["tp_execution"].astype(str).replace("0 days ", "", regex=True)
    if timings_df is None:
        timings_df = result
    else:
        timings_df = timings_df._append(result, ignore_index=True)

In [66]:
total_results = []
for time_row in timings_df.itertuples():
    _, algorithm, runinfo, date = time_row.tp_identifier[:-4].split("-",3)
    dataset = DATASET[runinfo[0]]
    runid = runinfo[1:]

    for run_row in runnings_df.itertuples():
        run_id, run_date = run_row.identifier.split("-",3)[2:]
        
        if runid == run_id and run_row.algorithm == algorithm and run_row.dataset == dataset and date == run_date:
            total_results.append(run_row + time_row + (runid, run_id))
            break

resulting_df = pd.DataFrame(list(total_results), columns=
                            ["run_id", "creation", "execution", "deletion", "creation_sec",
                              "execution_sec", "deletion_sec", "total_sec", "identifier",
                              "date", "begin_tp", "end_tp", "dataset", "algorithm", "timing_id", "tp_dataframe", 
                              "tp_execution", "tp_dataframe_sec", "tp_execution_sec", "date", 
                              "begin", "end", "tp_identifier", "A", "B"])

In [67]:
resulting_df

run_id         creation        execution         deletion  creation_sec  \
0        0  00:00:47.134556  01:24:38.728344  00:00:39.055283            47   
1        1  00:00:06.354604  02:14:09.112585  00:00:46.430160             6   
2        2  00:00:06.259021  01:17:57.328310  00:00:40.111874             6   
3        3  00:00:06.217704  02:56:15.749721  00:00:40.724346             6   
4        4  00:00:06.276408  01:19:50.846033  00:00:41.059981             6   
..     ...              ...              ...              ...           ...   
76      83  00:00:06.377283  05:25:50.633033  00:00:40.699194             6   
77      84  00:00:09.141037  06:18:26.372392  00:00:45.936919             9   
78      85  00:00:06.366347  06:50:16.817970  00:00:40.289671             6   
79      86  00:00:06.332181  06:42:04.481527  00:00:40.594852             6   
80      87  00:00:06.498242  06:21:50.322391  00:00:40.473011             6   

    execution_sec  deletion_sec  total_sec                 identifier  \
0            5078            39       5164   tenantb-lpa-0-2023-07-14   
1            8049            46       8101   tenantb-lpa-1-2023-07-13   
2            4677            40       4723   tenantb-lpa-1-2023-07-14   
3           10575            40      10621  tenantb-lpa-18-2023-07-20   
4            4790            41       4837  tenantb-lpa-19-2023-07-20   
..            ...           ...        ...                        ...   
76          19550            40      19596   tenantc-tc-13-2023-07-12   
77          22706            45      22760   tenantc-tc-13-2023-07-19   
78          24616            40      24662   tenantc-tc-14-2023-07-20   
79          24124            40      24170   tenantc-tc-15-2023-07-20   
80          22910            40      22956   tenantc-tc-16-2023-07-20   

          date  ...     tp_dataframe     tp_execution tp_dataframe_sec  \
0   2023-07-14  ...  00:01:43.937649  01:21:47.565309              103   
1   2023-07-13  ...  00:02:12.987587  02:11:04.291901              132   
2   2023-07-14  ...  00:01:45.001522  01:15:29.495627              105   
3   2023-07-20  ...  00:01:42.686720  02:54:20.515146              102   
4   2023-07-20  ...  00:01:42.368687  01:17:59.035754              102   
..         ...  ...              ...              ...              ...   
76  2023-07-12  ...  00:06:14.414101  05:18:43.892726              374   
77  2023-07-19  ...  00:07:33.005993  06:10:39.770562              453   
78  2023-07-19  ...  00:08:32.705704  06:40:50.701144              512   
79  2023-07-20  ...  00:07:28.249002  06:34:24.844752              448   
80  2023-07-20  ...  00:05:55.759517  06:15:27.557675              355   

   tp_execution_sec        date            begin              end  \
0              4907  2023-07-14  08:39:27.468859  10:02:58.971817   
1              7864  2023-07-13  14:45:33.364353  16:58:50.643841   
2              4529  2023-07-14  11:45:41.833517  13:02:56.330666   
3             10460  2023-07-20  18:35:04.689820  21:31:07.891686   
4              4679  2023-07-20  21:32:48.442117  22:52:29.846558   
..              ...         ...              ...              ...   
76            19123  2023-07-12  14:43:29.870623  20:08:28.177450   
77            22239  2023-07-19  15:49:37.737614  22:07:50.514169   
78            24050  2023-07-19  22:09:44.139333  04:59:07.546181   
79            23664  2023-07-20  05:00:33.140914  11:42:26.234668   
80            22527  2023-07-20  11:48:51.422603  18:10:14.739795   

                     tp_identifier   A   B  
0    timings-lpa-b0-2023-07-14.txt   0   0  
1    timings-lpa-b1-2023-07-13.txt   1   1  
2    timings-lpa-b1-2023-07-14.txt   1   1  
3   timings-lpa-b18-2023-07-20.txt  18  18  
4   timings-lpa-b19-2023-07-20.txt  19  19  
..                             ...  ..  ..  
76   timings-tc-c13-2023-07-12.txt  13  13  
77   timings-tc-c13-2023-07-19.txt  13  13  
78   timings-tc-c14-2023-07-20.txt  14  14  
79   timin

In [68]:
tp_start_info, tp_start_sec_info = [],[]
for _ in zip(resulting_df["begin_tp"], resulting_df["begin"]):
    tp_start = date_format(_[1]) - date_format(_[0])
    if "day" in str(tp_start):
        tp_start = tp_start + timedelta(days=-int(str(tp_start).split("day")[0]))
    tp_start_sec = int(tp_start.total_seconds())
    tp_start_info.append(tp_start)
    tp_start_sec_info.append(tp_start_sec)
resulting_df["tp_start"] = tp_start_info
resulting_df["tp_start_sec"] = tp_start_sec_info
resulting_df["tp_start"] = resulting_df["tp_start"].astype(str).replace("0 days ", "", regex=True)

In [69]:
resulting_df["execution_remainder"] = resulting_df["execution_sec"] - resulting_df["tp_start_sec"] - resulting_df["tp_dataframe_sec"] - resulting_df["tp_execution_sec"]
resulting_df['execution_remainder'] = resulting_df['execution_remainder'].where(resulting_df['execution_remainder'] >= 0, 0)

In [70]:
resulting_df.groupby(["algorithm", "dataset"]).describe()

run_id                                                  \
                     count  mean       std   min    25%   50%    75%   max   
algorithm dataset                                                            
lpa       kgs         10.0  51.5  3.027650  47.0  49.25  51.5  53.75  56.0   
          wiki-talk   10.0   5.0  4.082483   0.0   2.25   4.5   6.75  14.0   
pr        kgs         10.0  62.5  3.027650  58.0  60.25  62.5  64.75  67.0   
          wiki-talk   10.0  19.5  3.027650  15.0  17.25  19.5  21.75  24.0   
sp        kgs         10.0  72.5  3.027650  68.0  70.25  72.5  74.75  77.0   
          wiki-talk   10.0  29.5  3.027650  25.0  27.25  29.5  31.75  34.0   
tc        kgs         10.0  82.5  3.027650  78.0  80.25  82.5  84.75  87.0   
          wiki-talk   11.0  40.0  3.316625  35.0  37.50  40.0  42.50  45.0   

                    creation_sec        ... tp_start_sec        \
                           count  mean  ...          75%   max   
algorithm dataset                       ...                      
lpa       kgs               10.0   9.6  ...        18.50  95.0   
          wiki-talk         10.0  10.1  ...        38.75  66.0   
pr        kgs               10.0   6.1  ...        13.75  14.0   
          wiki-talk         10.0   6.2  ...        11.00  14.0   
sp        kgs               10.0   6.3  ...        12.75  19.0   
          wiki-talk         10.0   6.1  ...        13.75  16.0   
tc        kgs               10.0  18.4  ...        50.75  58.0   
          wiki-talk         11.0   6.0  ...        12.50  30.0   

                    execution_remainder                                        \
                                  count       mean         std  min  25%  50%   
algorithm dataset                                                               
lpa       kgs                      10.0   1.800000    0.421637  1.0  2.0  2.0   
          wiki-talk                10.0   1.400000    0.699206  0.0  1.0  1.5   
pr        kgs                      10.0   0.700000    0.823273  0.0  0.0  0.5   
          wiki-talk                10.0   0.600000    0.516398  0.0  0.0  1.0   
sp        kgs                      10.0   0.800000    0.421637  0.0  1.0  1.0   
          wiki-talk                10.0   0.800000    0.788811  0.0  0.0  1.0   
tc        kgs                      10.0   2.300000    0.674949  1.0  2.0  2.0   
          wiki-talk                11.0  55.272727  177.682352  0.0  1.5  2.0   

                                 
                     75%    max  
algorithm dataset                
lpa       kgs        2.0    2.0  
          wiki-talk  2.0    2.0  
pr        kgs        1.0    2.0  
          wiki-talk  1.0    1.0  
sp        kgs        1.0    1.0  
          wiki-talk  1.0    2.0  
tc        kgs        3.0    3.0  
          wiki-talk  2.0  591.0  

[8 rows x 80 columns]

In [71]:
resulting_df = resulting_df.drop_duplicates('run_id', keep=False)

In [72]:
resulting_df.groupby(["algorithm", "dataset"]).describe()

run_id                                                  \
                     count  mean       std   min    25%   50%    75%   max   
algorithm dataset                                                            
lpa       kgs         10.0  51.5  3.027650  47.0  49.25  51.5  53.75  56.0   
          wiki-talk   10.0   5.0  4.082483   0.0   2.25   4.5   6.75  14.0   
pr        kgs         10.0  62.5  3.027650  58.0  60.25  62.5  64.75  67.0   
          wiki-talk   10.0  19.5  3.027650  15.0  17.25  19.5  21.75  24.0   
sp        kgs         10.0  72.5  3.027650  68.0  70.25  72.5  74.75  77.0   
          wiki-talk   10.0  29.5  3.027650  25.0  27.25  29.5  31.75  34.0   
tc        kgs         10.0  82.5  3.027650  78.0  80.25  82.5  84.75  87.0   
          wiki-talk   11.0  40.0  3.316625  35.0  37.50  40.0  42.50  45.0   

                    creation_sec        ... tp_start_sec        \
                           count  mean  ...          75%   max   
algorithm dataset                       ...                      
lpa       kgs               10.0   9.6  ...        18.50  95.0   
          wiki-talk         10.0  10.1  ...        38.75  66.0   
pr        kgs               10.0   6.1  ...        13.75  14.0   
          wiki-talk         10.0   6.2  ...        11.00  14.0   
sp        kgs               10.0   6.3  ...        12.75  19.0   
          wiki-talk         10.0   6.1  ...        13.75  16.0   
tc        kgs               10.0  18.4  ...        50.75  58.0   
          wiki-talk         11.0   6.0  ...        12.50  30.0   

                    execution_remainder                                        \
                                  count       mean         std  min  25%  50%   
algorithm dataset                                                               
lpa       kgs                      10.0   1.800000    0.421637  1.0  2.0  2.0   
          wiki-talk                10.0   1.400000    0.699206  0.0  1.0  1.5   
pr        kgs                      10.0   0.700000    0.823273  0.0  0.0  0.5   
          wiki-talk                10.0   0.600000    0.516398  0.0  0.0  1.0   
sp        kgs                      10.0   0.800000    0.421637  0.0  1.0  1.0   
          wiki-talk                10.0   0.800000    0.788811  0.0  0.0  1.0   
tc        kgs                      10.0   2.300000    0.674949  1.0  2.0  2.0   
          wiki-talk                11.0  55.272727  177.682352  0.0  1.5  2.0   

                                 
                     75%    max  
algorithm dataset                
lpa       kgs        2.0    2.0  
          wiki-talk  2.0    2.0  
pr        kgs        1.0    2.0  
          wiki-talk  1.0    1.0  
sp        kgs        1.0    1.0  
          wiki-talk  1.0    2.0  
tc        kgs        3.0    3.0  
          wiki-talk  2.0  591.0  

[8 rows x 80 columns]

In [73]:
df_corrected = resulting_df[resulting_df["execution_remainder"] < 10] # remove problem executions

In [74]:
for _ in TESTS:
    __ = df_corrected[df_corrected.identifier.str.contains(_)]
    __.columns = [c.replace("_", "") for c in __.columns]
    __.to_csv(f"result_v2/clean/clean-{_}.csv")

In [75]:
for _ in TESTS:
    selected_combined = ["creation_sec", "tp_start_sec", "tp_dataframe_sec", "tp_execution_sec", "execution_remainder", "deletion_sec"]
    combined = df_corrected[df_corrected.identifier.str.contains(_)][selected_combined]
    combined = combined.reset_index()
    combined = combined[selected_combined]

    if len(combined) == 0:
        continue

    data = "wiki-talk" if _.split("-")[0][-1:] == "b" else "kgs"
    algorithm = "pr" if _.split("-")[1] == "pagerank" else _.split("-")[1]

    ax = combined.plot(kind="bar", stacked=True, legend=False)
    # Add Title and Labels
    plt.title(f"Total execution time in seconds of {data} - {algorithm}")
    plt.xlabel("Run ID")
    plt.ylabel("Time (s)")
    for i, item in enumerate(reversed(ax.containers)):
        if i == 0:
            ax.bar_label(item, color="black", weight="bold")#, rotation=90, padding=2)
    plt.tight_layout()
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles[::-1], labels[::-1], loc=(1.015,0.6))
    plt.tight_layout()
    plt.savefig(f"result_v2/bar/{_}-bar.png")
    plt.close()

    df = df_corrected[df_corrected.identifier.str.contains(_)][["total_sec"]]
    ax3 = df.boxplot(column="total_sec",rot=90)
    # # Add Title and Labels
    plt.title(f"Boxplot of {data} - {algorithm}")
    plt.xlabel("Part of total execution")
    plt.ylabel("Time (s)")
    plt.tight_layout()

    plt.tight_layout()
    plt.savefig(f"result_v2/box/{_}-box.png")
    plt.close()

In [76]:
for _ in TESTS:
    print(_)
    rows_df = df_corrected[df_corrected.identifier.str.contains(_)]
    print(shapiro(rows_df["total_sec"].tolist()))



    # try:
    #     _, pnorm = mannwhitneyu(res["1_default_prometheus"], res["2_final_settings"], method="asymptotic")
    #     print(pnorm)
    # except Exception as e:
    #     print(e)

    print(rows_df["total_sec"].describe())

tenantb-pagerank
ShapiroResult(statistic=0.902397871017456, pvalue=0.23273929953575134)
count     10.000000
mean     221.700000
std        4.715224
min      214.000000
25%      218.500000
50%      223.500000
75%      224.750000
max      227.000000
Name: total_sec, dtype: float64
tenantb-sp
ShapiroResult(statistic=0.8809004426002502, pvalue=0.13363558053970337)
count     10.000000
mean     220.100000
std        1.852926
min      216.000000
25%      220.000000
50%      220.000000
75%      220.750000
max      223.000000
Name: total_sec, dtype: float64
tenantc-pr
ShapiroResult(statistic=0.957907497882843, pvalue=0.7617748975753784)
count     10.000000
mean     584.300000
std       29.765939
min      541.000000
25%      560.250000
50%      588.500000
75%      605.500000
max      631.000000
Name: total_sec, dtype: float64
tenantc-sp
ShapiroResult(statistic=0.7527475953102112, pvalue=0.0038545257411897182)
count     10.000000
mean     505.300000
std       28.193971
min      481.000000
25%    